In [43]:
from ipywidgets import Layout, Button, Box, FloatText,Text, Password, Dropdown, Label, IntSlider,DatePicker
from IPython.display import clear_output
import datetime


trading_pairs_in = Text() #['AKT/USDT','CRWNY/USDT','ABBC/USDT','ADEL/USDT','BOLT/USDT','CAPS/USDT','CET/USDT','CNTR/USDT','D11/USDT','DIVI/USDT','EZ/USDT','FINE/USDT','FIN/USDT','GT/USDT','HPB/USDT','LUSD/USDT','MAHA/USDT','MITX/USDT','NIOX/USDT','OIN/USDT','PKF/USDT','PLN/USDT','PRXY/USDT','REVV/USDT','STF/USDT','TARA/USDT','ATLAS/USDT','CPOOL/USDT','GARI/USDT','JET/USDT','LOOKS/USDT','MONI/USDT','NEWO/USDT','RNDR/USDT','TIME/USDT','VERA/USDT']
api_key_in = Password()
api_secret_in = Password()
api_passphrase_in = Password()
api_group_in = Text()
start_date_in = DatePicker(disabled=False,value=datetime.datetime.now())
end_date_in = DatePicker(disabled=False,value=datetime.datetime.now())

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_passphrase_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_group_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

box_passPhrase = Box([Label(value='Passphrase'), 
    api_passphrase_in], layout=op_item_passphrase_layout)
box_group =   Box([Label(value='Api group'), 
    api_group_in], layout=op_item_group_layout)

form_items = [
    Box([Label(value='Api Key'), 
         api_key_in], layout=form_item_layout),
    Box([Label(value='Secret Key'), 
         api_secret_in], layout=form_item_layout),
    box_passPhrase,
    box_group,
    Box([Label(value='Start Date'), start_date_in], layout=form_item_layout),
    Box([Label(value='End Date'), end_date_in], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

box_passPhrase.layout.display = 'none'
box_group.layout.display = 'flex'
display(form)

def get_exchange_data():

    if 'google.colab' in str(get_ipython()):
      !git clone https://github.com/trupa/pnl-analysis
      %cd pnl-analysis/

    import pandas as pd
    import numpy as np
    import json
    import plotly.graph_objects as go
    from IPython.core.display import display, HTML
    from datetime import datetime

    # Set display
    pd.options.display.float_format = '{:,.10f}'.format

    START_TIME=start_date_in.value
    END_TIME=end_date_in.value
    api_key = api_key_in.value
    api_secret = api_secret_in.value
    api_group = api_group_in.value
    client = None
    start_dt = int(pd.to_datetime(START_TIME).timestamp()*1000)
    end_dt = int(pd.to_datetime(END_TIME).timestamp()*1000)

    from src.ascendex.AscendexClientWrapper import AscendexClientWrapper
    client = AscendexClientWrapper.createInstance(api_key,api_secret,api_group)

    from jinja2 import Template
    
    class Printer:
        @staticmethod
        def h1(*title):
            title_t = Template("<h1>{{title}}</h1>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h2(*title):
            title_t = Template("<h2>{{title}}</h2>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h3(*title):
            title_t = Template("<h3>{{title}}</h3>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))

        @staticmethod
        def html(html):
            display(HTML(html))
        
        @staticmethod
        def p_df(df):
            """Neatly display a dataframe"""
            display(HTML(df.to_html()))
            
        def p_dict(dt):
            df = pd.DataFrame()
            df['_'] = dt.keys()
            df['__'] = dt.values()
            Printer.p_df(df)

    df_trades = client.get_all_trades(start_dt,end_dt)
    df_balances = client.get_all_asset_balances()

    return df_trades, df_balances


Box(children=(Box(children=(Label(value='Api Key'), Password()), layout=Layout(display='flex', flex_flow='row'…

In [ ]:
from src.processing import pnl_calculate
from src.processing import calc_trading_fees

df_trades, df_balances = get_exchange_data()

total_fee, commissions = calc_trading_fees(df_trades)